# Video Classifier Using CNN and RNN
#!dir

In [ ]:
bro dini badulu jupyter notebook lo chesaddam ra em antav ok

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir(r"C:\Users\gopiv\Downloads\Video-Classifier-Using-CNN-and-RNN-main\Video-Classifier-Using-CNN-and-RNN-main\datasets\train")

label_types = os.listdir(r"C:\Users\gopiv\Downloads\Video-Classifier-Using-CNN-and-RNN-main\Video-Classifier-Using-CNN-and-RNN-main\datasets\train")
print (label_types)

FileNotFoundError: ignored

# Preparing Training Data

In [ ]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir(r"C:\Users\gopiv\Downloads\Video-Classifier-Using-CNN-and-RNN-main\Video-Classifier-Using-CNN-and-RNN-main\datasets\train" + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str(r"C:\Users\gopiv\Downloads\Video-Classifier-Using-CNN-and-RNN-main\Video-Classifier-Using-CNN-and-RNN-main\datasets\train" + '/' +item) + '/' + room))

# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())


        tag                                         video_name
0  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
1  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
2  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
3  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
4  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
      tag                                         video_name
30  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
31  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
32  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
33  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
34  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...


In [ ]:
df = train_df.loc[:,['tag','video_name']]
df
df.to_csv('train.csv')

# Preparing Test Data

In [ ]:
dataset_path = os.listdir(r"C:\Users\gopiv\Downloads\Video-Classifier-Using-CNN-and-RNN-main\Video-Classifier-Using-CNN-and-RNN-main\datasets\test")
print(dataset_path)

room_types = os.listdir(r"C:\Users\gopiv\Downloads\Video-Classifier-Using-CNN-and-RNN-main\Video-Classifier-Using-CNN-and-RNN-main\datasets\test")
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir(r"C:\Users\gopiv\Downloads\Video-Classifier-Using-CNN-and-RNN-main\Video-Classifier-Using-CNN-and-RNN-main\datasets\test" + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str(r"C:\Users\gopiv\Downloads\Video-Classifier-Using-CNN-and-RNN-main\Video-Classifier-Using-CNN-and-RNN-main\datasets\test" + '/' +item) + '/' + room))

# Build a dataframe
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['tag','video_name']]
df
df.to_csv('test.csv')

['accident', 'fight']
Types of activities found:  2
        tag                                         video_name
0  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
1  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
2  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
3  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
4  accident  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
     tag                                         video_name
5  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
6  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
7  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
8  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...
9  fight  C:\Users\gopiv\Downloads\Video-Classifier-Usin...


In [ ]:
!pip install tensorflow_docs

     -------------------------------------- 183.6/183.6 kB 5.6 MB/s eta 0:00:00


In [ ]:
!pip install imutils
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

# Data preparation

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 35
Total videos for testing: 10


,Unnamed: 0,video_name,tag
5,5,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,accident
3,3,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,accident
21,21,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,fight
8,8,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,accident
16,16,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,accident
34,34,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,fight
9,9,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,accident
27,27,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,fight
23,23,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,fight
28,28,C:\Users\gopiv\Downloads\Video-Classifier-Usin...,fight


# Feed the videos to a network:


In [ ]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

   ### Feature Extraction

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 16s 0us/step


### Label Encoding
StringLookup layer encode the class labels as integers.

In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['accident', 'fight']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int64)

Finally, we can put all the pieces together to create our data processing utility.

In [ ]:
#print(train_data[0].shape)
#train_data[0]

In [ ]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()

    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values

    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters

1/1 [==============================] - 0s 63ms/step
Frame features in train set: (35, 20, 2048)
Frame masks in train set: (35, 20)
train_labels in train set: (35, 1)
test_labels in train set: (10, 1)


# The sequence model
Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 0.8006 - accuracy: 0.4583
Epoch 1: val_loss improved from inf to 0.87677, saving model to ./tmp\video_classifier
1/1 [==============================] - 10s 10s/step - loss: 0.8006 - accuracy: 0.4583 - val_loss: 0.8768 - val_accuracy: 0.0909
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 0.6361 - accuracy: 0.7083
Epoch 2: val_loss did not improve from 0.87677
1/1 [==============================] - 0s 50ms/step - loss: 0.6361 - accuracy: 0.7083 - val_loss: 0.9222 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - ETA: 0s - loss: 0.6018 - accuracy: 0.7500
Epoch 3: val_loss did not improve from 0.87677
1/1 [==============================] - 0s 56ms/step - loss: 0.6018 - accuracy: 0.7500 - val_loss: 0.9686 - val_accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - ETA: 0s - loss: 0.6093 - accuracy: 0.7083
Epoch 4: val_loss did not improve from 0.87677
1/1 [=======

# Inference

In [ ]:
import cv2

def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        label = class_vocab[i]
        confidence = probabilities[i] * 100
        print(f"  {label}: {confidence:5.2f}%")

    for i, frame in enumerate(frames):
        # Add label to each frame
        frame_label = class_vocab[np.argmax(probabilities)]
        frame_confidence = np.max(probabilities) * 100

        cv2.putText(frame, f"{frame_label}: {frame_confidence:.2f}%", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Frame', frame)

        key = cv2.waitKey(25) & 0xFF
        if key == ord('q'):
            break

    cv2.destroyAllWindows()
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

for frame in test_frames:
    cv2.namedWindow('Frame', cv2.WINDOW_FULLSCREEN)
    cv2.imshow('Frame', frame)

    key = cv2.waitKey(25) & 0xFF
    if key == ord('q'):
        break

cv2.destroyAllWindows()


Test video path: C:\Users\gopiv\Downloads\Video-Classifier-Using-CNN-and-RNN-main\Video-Classifier-Using-CNN-and-RNN-main\datasets\test/fight/pexels-cottonbro-studio-4761711-4096x2160-25fps.mp4
1/1 [==============================] - 0s 31ms/step
  accident: 59.46%
  fight: 40.54%
